In [1]:
#Import necessary libraries for ML and Pre-Processing

!pip install feature-engine 
!pip install imbalanced-learn

from feature_engine.imputation import RandomSampleImputer


#Importing the necessary libraries for EDA and model building

import numpy as np 

import pandas as pd 

from sklearn import preprocessing

import matplotlib.pyplot as plt 

import seaborn as sns

import warnings

import time

warnings.simplefilter(action='ignore')



import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split


#importing ML models from sklearn library

from sklearn.naive_bayes import GaussianNB

from sklearn.neighbors import KNeighborsClassifier

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.linear_model import LogisticRegression

from sklearn.svm import SVC

from sklearn.ensemble import RandomForestClassifier

from sklearn.ensemble import GradientBoostingClassifier

from sklearn.tree import DecisionTreeClassifier

from sklearn.linear_model import RidgeClassifier

from sklearn.linear_model import LogisticRegression



#Importing metrics functions from SK Learn

from sklearn.metrics import roc_auc_score, r2_score, mean_absolute_error, mean_squared_error

from sklearn.model_selection import cross_val_score

from sklearn.metrics import classification_report, confusion_matrix



from sklearn.decomposition import PCA

from sklearn.preprocessing import StandardScaler

from sklearn import metrics

from pprint import pprint


# # Used for Downloading MNIST

# from sklearn.datasets import fetch_mldata



# Used for Splitting Training and Test Sets

from sklearn.model_selection import train_test_split



%matplotlib inline

In [2]:
df = pd.read_csv('bodyPerformance.csv')


# set random seed for reproducibility
np.random.seed(42)
df.duplicated().sum()

print(f'with duplicates:{df.shape}')
df.drop_duplicates(inplace=True)
print(f'without duplicates:{df.shape}')

df.rename(columns={"height_cm": "height", "weight_kg": "weight","body fat_%":"body_fat",
                   "sit and bend forward_cm":"sit_and_bend_forward","sit-ups counts":"sit-ups",
                   "broad jump_cm":"broad_jump","class":"performance"},inplace=True)



df.age = df.age.astype(int)

gender = {'F':0,'M':1}
df['gender'] =df['gender'].replace(gender)

df["BMI_val"]=round((df['weight']/(df['height'])/(df['height']))*10000,2)

def BMI (row):
    bmi = ((row.weight/row.height)/row.height)*10000
    if bmi>=18.5 and bmi < 25: 
        return 0 
    elif bmi < 18.5:
        return 1
    elif bmi >= 25 and bmi < 30:
        return 2
    else:
        return 3
    
df['BMI'] = df.apply(BMI,axis=1)



Class = {'A':0, 'B':1, 'C':2, 'D':3}
df['class'] =df['performance'].replace(Class)




df = df[(df["body_fat"] <= 70)]
df = df[(df["sit_and_bend_forward"] <= 50)]
df = df[(df["broad_jump"] >0)]
df = df[df['diastolic'] < df['systolic']]
df = df[df['diastolic'] >=10]

df.drop(['height','weight','performance'],axis=1, inplace=True)

df["MAP"]=round((((2*df['diastolic'])+(df['systolic']))/3),2)
df.drop(['diastolic','systolic'],axis=1, inplace=True)


y = df['class']
df.drop(['class','BMI'], axis=1, inplace=True)
X = df

train_feature, test_feature, train_label, test_label = train_test_split(X, y, test_size=0.2, random_state=42)

train_feature.head()


with duplicates:(13393, 12)
without duplicates:(13392, 12)


,age,gender,body_fat,gripForce,sit_and_bend_forward,sit-ups,broad_jump,BMI_val,MAP
415,25,1,20.5,54.7,28.5,38.0,217.0,26.23,103.33
9229,60,1,10.6,55.9,20.6,43.0,198.0,21.16,110.67
6124,63,1,22.8,33.3,24.2,30.0,191.0,24.87,105.67
10788,37,1,14.2,48.9,10.9,48.0,242.0,25.54,108.00
6532,46,1,16.5,58.5,15.4,46.0,225.0,24.45,105.00


# advanced analysis

In [3]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


models = {
    'Random Forest': RandomForestClassifier(),
    'XGBoost': XGBClassifier(),
    'SVC': SVC(probability=True),
    'LDA': LinearDiscriminantAnalysis(),
    'Multiple Logistic': LogisticRegression(solver='liblinear', multi_class='auto'),
    'KNN':KNeighborsClassifier(),
    'GausianNB':GaussianNB(),
    'Gradient Boost':GradientBoostingClassifier(criterion = "friedman_mse"),
    'Ridge': LogisticRegression(solver='saga', multi_class='multinomial',penalty='l2', C=1.0),
    'Lasso': LogisticRegression(solver='saga', multi_class='multinomial',penalty='l1', C=1),
    'Elastic-net':LogisticRegression(penalty='elasticnet', l1_ratio=0.5, C=1, solver='saga', multi_class='multinomial')



}

for name, model in models.items():
    model.fit(train_feature, train_label)
    
    # Training set
    train_pred = model.predict(train_feature)
    train_acc = accuracy_score(train_label, train_pred)
    train_err = 1 - train_acc
    train_f1 = f1_score(train_label, train_pred, average='weighted')
    train_cm = confusion_matrix(train_label, train_pred)
    
    # Test set
    test_pred = model.predict(test_feature)
    test_acc = accuracy_score(test_label, test_pred)
    test_err = 1 - test_acc
    test_f1 = f1_score(test_label, test_pred, average='weighted')
    test_cm = confusion_matrix(test_label, test_pred)
    
    print(name)
    print('='*len(name))
    print(f'Training Accuracy: {train_acc:.4f}')
    print(f'Training Error Rate: {train_err:.4f}')
    print(f'Training F1 Score: {train_f1:.4f}')
    print(f'Training Confusion Matrix:\n{train_cm}')
    print('\n')
    print(f'Test Accuracy: {test_acc:.4f}')
    print(f'Test Error Rate: {test_err:.4f}')
    print(f'Test F1 Score: {test_f1:.4f}')
    print(f'Test Confusion Matrix:\n{test_cm}')
    print('\n\n')


Random Forest
Training Accuracy: 1.0000
Training Error Rate: 0.0000
Training F1 Score: 1.0000
Training Confusion Matrix:
[[2699    0    0    0]
 [   0 2667    0    0]
 [   0    0 2693    0]
 [   0    0    0 2638]]


Test Accuracy: 0.7525
Test Error Rate: 0.2475
Test F1 Score: 0.7523
Test Confusion Matrix:
[[567  65  13   0]
 [152 417  81  26]
 [ 59 123 451  18]
 [ 11  37  77 578]]



XGBoost
Training Accuracy: 0.9424
Training Error Rate: 0.0576
Training F1 Score: 0.9425
Training Confusion Matrix:
[[2674   20    5    0]
 [ 209 2420   35    3]
 [  99  154 2440    0]
 [  15   27   49 2547]]


Test Accuracy: 0.7600
Test Error Rate: 0.2400
Test F1 Score: 0.7596
Test Confusion Matrix:
[[576  62   7   0]
 [151 424  82  19]
 [ 65 127 441  18]
 [  8  40  63 592]]



SVC
===
Training Accuracy: 0.6041
Training Error Rate: 0.3959
Training F1 Score: 0.6040
Training Confusion Matrix:
[[2007  662   30    0]
 [ 769 1127  703   68]
 [ 336  565 1456  336]
 [  79  196  491 1872]]


Test Accuracy: 0.6052


In [4]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


models = {
    'XGBoost': XGBClassifier(alpha=1)

}

for name, model in models.items():
    model.fit(train_feature, train_label)
    
    # Training set
    train_pred = model.predict(train_feature)
    train_acc = accuracy_score(train_label, train_pred)
    train_err = 1 - train_acc
    train_f1 = f1_score(train_label, train_pred, average='weighted')
    train_cm = confusion_matrix(train_label, train_pred)
    
    # Test set
    test_pred = model.predict(test_feature)
    test_acc = accuracy_score(test_label, test_pred)
    test_err = 1 - test_acc
    test_f1 = f1_score(test_label, test_pred, average='weighted')
    test_cm = confusion_matrix(test_label, test_pred)
    
    print(name)
    print('='*len(name))
    print(f'Training Accuracy: {train_acc:.4f}')
    print(f'Training Error Rate: {train_err:.4f}')
    print(f'Training F1 Score: {train_f1:.4f}')
    print(f'Training Confusion Matrix:\n{train_cm}')
    print('\n')
    print(f'Test Accuracy: {test_acc:.4f}')
    print(f'Test Error Rate: {test_err:.4f}')
    print(f'Test F1 Score: {test_f1:.4f}')
    print(f'Test Confusion Matrix:\n{test_cm}')
    print('\n\n')


XGBoost
Training Accuracy: 0.9476
Training Error Rate: 0.0524
Training F1 Score: 0.9477
Training Confusion Matrix:
[[2668   25    6    0]
 [ 201 2439   22    5]
 [  95  122 2476    0]
 [  20   24   41 2553]]


Test Accuracy: 0.7637
Test Error Rate: 0.2363
Test F1 Score: 0.7637
Test Confusion Matrix:
[[576  58  10   1]
 [147 433  77  19]
 [ 63 126 447  15]
 [  8  43  65 587]]





# Randomized grid search cv

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

# Create the Random Forest model
rf = RandomForestClassifier()

# Define the parameter grid to search over
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 4, 6],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

# Perform a grid search over the parameter grid using 5-fold cross-validation
grid_search = RandomizedSearchCV(estimator=rf, param_distributions=param_grid, n_iter=100, cv=5, 
                               scoring='accuracy', n_jobs=-1, random_state=42)
grid_search.fit(train_feature, train_label)

# Print the best hyperparameters and corresponding cross-validation score
print("Best hyperparameters: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)

# Use the best hyperparameters to fit the model on the training set
rf_best = RandomForestClassifier(n_estimators=grid_search.best_params_['n_estimators'], 
                                  max_depth=grid_search.best_params_['max_depth'], 
                                  min_samples_split=grid_search.best_params_['min_samples_split'],
                                  min_samples_leaf=grid_search.best_params_['min_samples_leaf'],
                                  max_features=grid_search.best_params_['max_features'])
rf_best.fit(train_feature, train_label)

# Evaluate the training and test accuracy of the model
train_acc = rf_best.score(train_feature, train_label)
test_acc = rf_best.score(test_feature, test_label)

print("Training accuracy: ", train_acc)
print("Test accuracy: ", test_acc)


Best hyperparameters:  {'n_estimators': 300, 'min_samples_split': 6, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 20}
Best cross-validation score:  0.740862443363787
Training accuracy:  0.9681219033373843
Test accuracy:  0.7454205607476635


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from sklearn.base import clone


# Define the model with bootstrapping
model = RandomForestClassifier(n_estimators=200, 
                               max_features='sqrt', 
                               min_samples_leaf= 1,
                               min_samples_split= 6,
                               max_depth=30, 
                               random_state=42)

# Train the model with bootstrapping
n_bootstraps = 10
bootstrapped_models = []
for i in range(n_bootstraps):
    X_boot, y_boot = resample(train_feature, train_label, random_state=i)
    model_i = clone(model)
    model_i.fit(X_boot, y_boot)
    bootstrapped_models.append(model_i)

# Make predictions by averaging the bootstrapped models
y_pred = np.mean([model.predict(test_feature) for model in bootstrapped_models], axis=0)

In [5]:
from sklearn.metrics import accuracy_score, f1_score

# Calculate accuracy and f1 score on train set
train_pred = np.mean([model.predict(train_feature) for model in bootstrapped_models], axis=0)
train_pred = np.round(train_pred)
train_pred[train_pred < 0.5] = 0
train_pred[train_pred >= 0.5] = 1
train_acc = accuracy_score(train_label, train_pred)
train_f1 = f1_score(train_label, train_pred, average='weighted')

# Calculate accuracy and f1 score on test set
test_pred = np.mean([model.predict(test_feature) for model in bootstrapped_models], axis=0)
test_pred = np.round(test_pred)
test_pred[test_pred < 0.5] = 0
test_pred[test_pred >= 0.5] = 1
test_acc = accuracy_score(test_label, test_pred)
test_f1 = f1_score(test_label, test_pred, average='weighted')

print("Train accuracy:", train_acc)
print("Train F1 score:", train_f1)
print("Test accuracy:", test_acc)
print("Test F1 score:", test_f1)



Train accuracy: 0.4804150696456951
Train F1 score: 0.3597388995177196
Test accuracy: 0.40523364485981306
Test F1 score: 0.2935536401082983


In [53]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# set random seed for reproducibility
np.random.seed(42)

# Create the XGBoost model
xgb = XGBClassifier(alpha=1)

# Define the parameter grid to search over
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'learning_rate': [0.1, 0.01, 0.001],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.2]
}

# Perform a grid search over the parameter grid using 5-fold cross-validation
grid_search = grid_search = RandomizedSearchCV(estimator=xgb, param_distributions=param_grid, n_iter=100, cv=5, 
                               scoring='accuracy', n_jobs=-1, random_state=42)
grid_search.fit(train_feature, train_label)

# Print the best hyperparameters and corresponding cross-validation score
print("Best hyperparameters: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)

# Use the best hyperparameters to fit the model on the training set
xgb_best = XGBClassifier(n_estimators=grid_search.best_params_['n_estimators'], 
                         max_depth=grid_search.best_params_['max_depth'], 
                         learning_rate=grid_search.best_params_['learning_rate'],
                         subsample=grid_search.best_params_['subsample'],
                         colsample_bytree=grid_search.best_params_['colsample_bytree'],
                         gamma=grid_search.best_params_['gamma'],
                        alpha=1)
xgb_best.fit(train_feature, train_label)

# Evaluate the training and test accuracy of the model
train_acc = xgb_best.score(train_feature, train_label)
test_acc = xgb_best.score(test_feature, test_label)

print("Training accuracy: ", train_acc)
print("Test accuracy: ", test_acc)


Best hyperparameters:  {'subsample': 1.0, 'n_estimators': 200, 'max_depth': 10, 'learning_rate': 0.1, 'gamma': 0.1, 'colsample_bytree': 0.8}
Best cross-validation score:  0.7486215499425446
Training accuracy:  0.9987847059923343
Test accuracy:  0.7585046728971963


In [54]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Create the Gradient Boosting model
gb = GradientBoostingClassifier()

# Define the parameter grid to search over
param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.05, 0.1, 0.15],
    'subsample': [0.6, 0.8, 1.0],
    'min_samples_split': [2, 4, 6],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None],
    'random_state': [42],
    
}

# Perform a grid search over the parameter grid using 5-fold cross-validation
grid_search = RandomizedSearchCV(estimator=gb, param_distributions=param_grid, n_iter=100, cv=5, 
                               scoring='accuracy', n_jobs=-1, random_state=42)
grid_search.fit(train_feature, train_label)

# Print the best hyperparameters and corresponding cross-validation score
print("Best hyperparameters: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)

# Use the best hyperparameters to fit the model on the training set
gb_best = GradientBoostingClassifier(n_estimators=grid_search.best_params_['n_estimators'], 
                                      max_depth=grid_search.best_params_['max_depth'], 
                                      learning_rate=grid_search.best_params_['learning_rate'],
                                     subsample= grid_search.best_params_['subsample'],
                                     min_samples_split= grid_search.best_params_['min_samples_split'],
                                     min_samples_leaf =grid_search.best_params_['min_samples_leaf'] ,
                                     max_features =grid_search.best_params_['max_features'] ,
                                     random_state =grid_search.best_params_['random_state'] 
                                    )
gb_best.fit(train_feature, train_label)

# Evaluate the training and test accuracy of the model
train_acc = gb_best.score(train_feature, train_label)
test_acc = gb_best.score(test_feature, test_label)

print("Training accuracy: ", train_acc)
print("Test accuracy: ", test_acc)


Best hyperparameters:  {'subsample': 1.0, 'random_state': 42, 'n_estimators': 100, 'min_samples_split': 4, 'min_samples_leaf': 4, 'max_features': None, 'max_depth': 7, 'learning_rate': 0.1}
Best cross-validation score:  0.7405823316861315
Training accuracy:  0.9684958399551276
Test accuracy:  0.7577570093457944


In [55]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Create the LDA model
lda = LinearDiscriminantAnalysis()

# Define the parameter grid to search over
param_grid = {
    'solver': ['svd', 'lsqr', 'eigen'],
    'shrinkage': [None, 'auto', 0.1, 0.5, 0.9],
    'n_components': [None, 1, 2, 3]
}

# Perform a grid search over the parameter grid using 5-fold cross-validation
grid_search = RandomizedSearchCV(estimator=lda, param_distributions=param_grid, n_iter=100, cv=5, 
                               scoring='accuracy', n_jobs=-1, random_state=42)
grid_search.fit(train_feature, train_label)

# Print the best hyperparameters and corresponding cross-validation score
print("Best hyperparameters: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)

# Use the best hyperparameters to fit the model on the training set
lda_best = LinearDiscriminantAnalysis(solver=grid_search.best_params_['solver'], 
                                      shrinkage=grid_search.best_params_['shrinkage'], 
                                      n_components=grid_search.best_params_['n_components'])
lda_best.fit(train_feature, train_label)

# Evaluate the training and test accuracy of the model
train_acc = lda_best.score(train_feature, train_label)
test_acc = lda_best.score(test_feature, test_label)

print("Training accuracy: ", train_acc)
print("Test accuracy: ", test_acc)


Best hyperparameters:  {'solver': 'lsqr', 'shrinkage': 'auto', 'n_components': None}
Best cross-validation score:  0.6119476740375667
Training accuracy:  0.6142843787977937
Test accuracy:  0.6130841121495327


In [56]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Create the KNN model
knn = KNeighborsClassifier()

# Define the parameter grid to search over
param_grid = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

# Perform a grid search over the parameter grid using 5-fold cross-validation
grid_search = RandomizedSearchCV(estimator=knn, param_distributions=param_grid, n_iter=100, cv=5, 
                               scoring='accuracy', n_jobs=-1, random_state=42)
grid_search.fit(train_feature, train_label)

# Print the best hyperparameters and corresponding cross-validation score
print("Best hyperparameters: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)

# Use the best hyperparameters to fit the model on the training set
knn_best = KNeighborsClassifier(n_neighbors=grid_search.best_params_['n_neighbors'], 
                                 weights=grid_search.best_params_['weights'], 
                                 p=grid_search.best_params_['p'])
knn_best.fit(train_feature, train_label)

# Evaluate the training and test accuracy of the model
train_acc = knn_best.score(train_feature, train_label)
test_acc = knn_best.score(test_feature, test_label)

print("Training accuracy: ", train_acc)
print("Test accuracy: ", test_acc)


Best hyperparameters:  {'weights': 'distance', 'p': 1, 'n_neighbors': 7}
Best cross-validation score:  0.5904451376964517
Training accuracy:  1.0
Test accuracy:  0.6063551401869159


In [ ]:
from sklearn.metrics import confusion_matrix

gb_best.fit(train_feature, train_label)
gb_best_predict = gb_best.predict(test_feature)

print("=== Confusion Matrix ===")
print(confusion_matrix(test_label, gb_best_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(test_label, gb_best_predict))

In [ ]:
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import time

# Define models
models = {
    "Random Forest": RandomForestClassifier(),
    "XGBoost": XGBClassifier(),
    "SVM": SVC(),
    "LDA": LDA(),
}

# Define parameter grids for GridSearchCV
param_grids = {
    "Random Forest": {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, 30],
        'min_samples_split': [2, 4, 6]
    },
    "XGBoost": {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 4, 5],
        'learning_rate': [0.1, 0.01, 0.001],
        'subsample': [0.5, 0.8, 1],
    },
    "SVM": {
        'C': [0.1, 1, 10],
        'gamma': ['scale', 'auto', 1, 0.1, 0.01],
        'kernel': ['linear', 'rbf', 'sigmoid']
    },
    "LDA": {
        'solver': ['svd', 'lsqr'],
        'tol': [1e-3, 1e-4, 1e-5],
        'shrinkage': [None, 'auto', 0, 0.5, 1],
        'n_components': [2, 3, 4]
    }
}

for name, model in models.items():
    print(f"Running {name}")
    print("=" * 20)
    
    # Cross-validation
    start_time = time.time()
    grid_search = GridSearchCV(model, param_grids[name], cv=5, scoring='accuracy', n_jobs=-1)
    grid_search.fit(train_feature, train_label)
    end_time = time.time()
    cv_time = end_time - start_time
    
    print("Best hyperparameters: ", grid_search.best_params_)
    print("Best cross-validation score: ", grid_search.best_score_)
    print(f"Time taken for cross-validation: {cv_time:.2f} seconds")
    
    # Use the best hyperparameters to fit the model on the training set
    start_time = time.time()
    best_model = model.set_params(**grid_search.best_params_)
    best_model.fit(train_feature, train_label)
    end_time = time.time()
    train_time = end_time - start_time
    
    # Evaluate the model on training set
    train_pred = best_model.predict(train_feature)
    train_acc = accuracy_score(train_label, train_pred)
    train_f1 = f1_score(train_label, train_pred, average='weighted')
    train_cm = confusion_matrix(train_label, train_pred)

    # Evaluate the model on test set
    start_time = time.time()
    test_pred = best_model.predict(test_feature)
    end_time = time.time()
    test_time = end_time - start_time
    test_acc = accuracy_score(test_label, test_pred)
    test_f1 = f1_score(test_label, test_pred, average='weighted')
    test_cm = confusion_matrix(test_label, test_pred)
    
   
